In [2]:
import glob
import json
import os
import datetime

import pandas as pd
pd.set_option('display.max_rows', 100)
import plotly.express as px

In [3]:
DATA_PATH = "../vwkommi"

json_files = glob.glob(f"{DATA_PATH}/*.json")

In [4]:
def load_file(path):
    file_name = os.path.basename(os.path.splitext(path)[0])
    date = file_name.split("_")[-1].split("T")[0]

    cars = []

    with open(path, "r") as f:
        json_string = json.loads(f.read())

    for komm_num, elems in json_string.items():
        car = {
            "Datum": datetime.datetime.strptime(date, "%Y-%m-%d"),
            "Kommissionsnummer": komm_num,
            "FIN": "",
            "Modell": "",
            "Farbcode": "",
            "Farbe": "",
            "Motor": "",
            "Modelljahr": "",
            "Ausstattung": "",
        }

        num_elems = len(elems)
        for num, elem in zip(range(num_elems), elems):
            if num == 0:
                try:
                    car['FIN'] = elem["vin"]
                except KeyError:
                    car['FIN'] = ""
                car['Modell'] = elem["modelName"].split(" ")[0]
                car['Variante'] = ' '.join(elem["modelName"].split(" ")[1:])
                car['Farbcode'] = elem["exteriorColor"]

            if num == 1:
                car['Motor']  = elem["engine"]
                car['Modelljahr']  = elem["modelYear"]
                try:
                    car['Farbe'] = elem["exteriorColorText"]
                except KeyError:
                    car['Farbe'] = ""

                try:
                    specifications = elem["specifications"]
                    
                    specs = []
                    for spec in specifications:
                        specs.append(spec["codeText"])
                    car['Ausstattung'] = specs
                except KeyError:
                    car['Ausstattung'] = ""

        cars.append(car)

    df = pd.DataFrame(cars)

    df = df.explode('Ausstattung')
    return df


In [5]:
dfs = []
num_files = len(json_files)
for file, num in zip(json_files, range(len(json_files))):
    print(f"Parsing {file} ({num}/{num_files})")
    dfs.append(load_file(file))

df = pd.concat(dfs)

Parsing ../vwkommi/output_AF_5000-9999_2022-04-09T10.57.49.json (0/74)
Parsing ../vwkommi/output_AF_5000-9999_2022-03-29T17.44.56.json (1/74)
Parsing ../vwkommi/output_AJ_0-9999_2022-03-05T15.23.25.json (2/74)
Parsing ../vwkommi/output_AJ_0-9999_2022-04-23T10.54.17.json (3/74)
Parsing ../vwkommi/output_AI_0-9999_2022-03-26T11.27.59.json (4/74)
Parsing ../vwkommi/output_AK_0-9999_2022-04-16T10.27.07.json (5/74)
Parsing ../vwkommi/output_AF_5000-9999_2022-02-20T09.09.46.json (6/74)
Parsing ../vwkommi/output_AG_0-9999_2022-03-19T10.44.38.json (7/74)
Parsing ../vwkommi/output_AJ_0-9999_2022-02-27T20.45.14.json (8/74)
Parsing ../vwkommi/output_AF_5000-9999_2022-04-16T10.27.07.json (9/74)
Parsing ../vwkommi/output_AG_0-9999_2022-02-26T10.48.24.json (10/74)
Parsing ../vwkommi/output_AL_0-2000_2022-04-02T11.22.53.json (11/74)
Parsing ../vwkommi/output_AJ_0-9999_2022-03-19T10.44.38.json (12/74)
Parsing ../vwkommi/output_AH_0-9999_2022-03-12T18.09.59.json (13/74)
Parsing ../vwkommi/output_AK_0-9

In [6]:
df = df[df["Modell"] != "Volkswagen"]
df["Kommissionsbereich"] = df["Kommissionsnummer"].str[:2]

In [7]:
model_df = df[['Datum', 'Modell', 'Kommissionsnummer']].groupby(['Datum', 'Modell']).nunique().reset_index()

In [8]:
fig = px.line(
    model_df,
    x="Datum",
    y="Kommissionsnummer",
    color="Modell",
    title='Anzahl Fahrzeuge Pro Woche nach Modell',
    labels={
        "Kommissionsnummer": "Anzahl",
    }
)

fig.write_html("../html/images/model_count.html", include_plotlyjs="cdn", full_html=False)

In [9]:
varianten_df = df[['Datum', 'Modell', "Variante", 'Kommissionsnummer']].groupby(['Datum', 'Modell', "Variante"]).nunique().reset_index()

In [10]:
fig = px.line(
    varianten_df,
    x="Datum",
    y="Kommissionsnummer",
    color="Variante",
    title='Anzahl Fahrzeuge Pro Woche nach Variante',
    facet_col="Modell",
    labels={
        "Kommissionsnummer": "Anzahl",
    }
)

fig.write_html("../html/images/variants_count.html", include_plotlyjs="cdn", full_html=False)

In [11]:
kommissionsbereich_df = df[['Datum', 'Modell', "Kommissionsbereich", 'Kommissionsnummer']].groupby(['Datum', 'Modell', "Kommissionsbereich"]).nunique().reset_index()

In [12]:
fig = px.line(
    kommissionsbereich_df,
    x="Datum",
    y="Kommissionsnummer",
    color="Kommissionsbereich",
    title='Anzahl Fahrzeuge Pro Woche nach Kommisionsnummerbereich',
    facet_col="Modell",
    labels={
        "Kommissionsnummer": "Anzahl",
    }
)

fig.write_html("../html/images/kommbereich_count.html", include_plotlyjs="cdn", full_html=False)

In [13]:
fin_df = df[['Datum', 'Modell', 'FIN']].groupby(['Datum', 'Modell']).nunique().reset_index()

In [14]:
fig = px.line(
    fin_df,
    x="Datum",
    y="FIN",
    color="Modell",
    title='Anzahl FINs Pro Woche nach Modell',
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/fin_count.html", include_plotlyjs="cdn", full_html=False)

In [15]:
fin_varianten_df = df[['Datum', 'Modell', "Variante", 'FIN']].groupby(['Datum', 'Modell', "Variante"]).nunique().reset_index()

In [16]:
fig = px.line(
    fin_varianten_df,
    x="Datum",
    y="FIN",
    color="Variante",
    title='Anzahl FINs Pro Woche nach Variante',
    facet_col="Modell",
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/fin_variants_count.html", include_plotlyjs="cdn", full_html=False)

In [17]:
bereich_varianten_df = df[['Datum', 'Modell', "Kommissionsbereich", 'FIN']].groupby(['Datum', 'Modell', "Kommissionsbereich"]).nunique().reset_index()

In [18]:
fig = px.line(
    bereich_varianten_df,
    x="Datum",
    y="FIN",
    color="Kommissionsbereich",
    title='Anzahl FINs Pro Woche nach Kommisionsnummerbereich',
    facet_col="Modell",
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/komm_range_count.html", include_plotlyjs="cdn", full_html=False)